# Installation

In [11]:
!pip install --upgrade pip

    100% |████████████████████████████████| 1.4MB 3.4MB/s 
  Found existing installation: pip 18.1
    Uninstalling pip-18.1:
      Successfully uninstalled pip-18.1


In [27]:
!pip install seqeval[gpu]

  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [16]:
!pip install i2b2_import

  Using cached https://files.pythonhosted.org/packages/b4/17/ee0138e4e44b37c031861824abb61a6a290f8883a61d490608cb8dfd0e39/i2b2_import-1.6.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5c/1c/6997288da181277a0c29bc39a5f9143ff20b8c99f2a7d059cfb55163e165/psycopg2-2.8.3.tar.gz
    ERROR: Complete output from command python setup.py egg_info:
    ERROR: running egg_info
    creating pip-egg-info/psycopg2.egg-info
    writing pip-egg-info/psycopg2.egg-info/PKG-INFO
    writing dependency_links to pip-egg-info/psycopg2.egg-info/dependency_links.txt
    writing top-level names to pip-egg-info/psycopg2.egg-info/top_level.txt
    writing manifest file 'pip-egg-info/psycopg2.egg-info/SOURCES.txt'
    
    Error: pg_config executable not found.
    
    pg_config is required to build psycopg2 from source.  Please add the directory
    containing pg_config to the $PATH or specify the full executable path with the
    option:
    
        python setup.py build_ext --pg-con

In [17]:
!pip show pytorch-pretrained-bert

Name: pytorch-pretrained-bert
Version: 0.6.2
Summary: PyTorch version of Google AI BERT model with script to load Google pre-trained models
Home-page: https://github.com/huggingface/pytorch-pretrained-BERT
Author: Thomas Wolf, Victor Sanh, Tim Rault, Google AI Language Team Authors, Open AI team Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /root/.pyenv/versions/3.6.7/lib/python3.6/site-packages
Requires: tqdm, numpy, requests, boto3, regex, torch
Required-by: flair


In [18]:
!pip install pytorch-pretrained-bert==0.4.0

     |████████████████████████████████| 51kB 629kB/s 
ERROR: flair 0.4.2 has requirement pytorch-pretrained-bert>=0.6.1, but you'll have pytorch-pretrained-bert 0.4.0 which is incompatible.
  Found existing installation: pytorch-pretrained-bert 0.6.2
    Uninstalling pytorch-pretrained-bert-0.6.2:
      Successfully uninstalled pytorch-pretrained-bert-0.6.2


In [19]:
!pip install git+https://github.com/IINemo/libact.git@seq

  Cloning https://github.com/IINemo/libact.git (to revision seq) to /tmp/pip-req-build-i0yb8va9
  Running command git clone -q https://github.com/IINemo/libact.git /tmp/pip-req-build-i0yb8va9
  Running command git checkout -b seq --track origin/seq
  Switched to a new branch 'seq'
  Branch seq set up to track remote branch seq from origin.
  Stored in directory: /tmp/pip-ephem-wheel-cache-px0vocik/wheels/43/f8/6d/9e686dbc90eb0c3a91ceee9e382e6ab85954d9a83459bced85
Successfully built libact


In [20]:
!pip install git+https://github.com/IINemo/active_learning_toolbox.git@seq

  Cloning https://github.com/IINemo/active_learning_toolbox.git (to revision seq) to /tmp/pip-req-build-4a5skxiv
  Running command git clone -q https://github.com/IINemo/active_learning_toolbox.git /tmp/pip-req-build-4a5skxiv
  Running command git checkout -b seq --track origin/seq
  Switched to a new branch 'seq'
  Branch seq set up to track remote branch seq from origin.
  Stored in directory: /tmp/pip-ephem-wheel-cache-7qayn8qj/wheels/96/c0/0a/e37cee1f7228329c68c9185d15c632f7837f1ef5a863116bd8
Successfully built actleto


In [1]:
# Initialization

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import sys

In [2]:
import torch

device = torch.device("cuda")
n_gpu = torch.cuda.device_count()
print(n_gpu)
for i in range(n_gpu):
    print(torch.cuda.get_device_name(i))

1
GeForce GTX 1080 Ti


In [3]:
torch.cuda.empty_cache()

In [4]:
import logging
logger = logging.getLogger('active_learning')

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

fhandler = logging.FileHandler(filename='../preprosessing/conll03_active_learning.log', mode='a')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)

handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(formatter)
logger.addHandler(handler)

logger.setLevel(logging.DEBUG)

In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import json

# Global parameters

In [6]:
CACHE_DIR = '../workdir/models/0.4.0'

MAX_LEN = 100
#BATCH_SIZE = 105
BATCH_SIZE = 50 
# MAX_LEN = 150
# BATCH_SIZE = 32
PRED_BATCH_SIZE = 1000
random_state = 2019

# Load dataset

In [13]:
from i2b2_utils import drop_noise_samples, tokenize_and_generate_labels, split_train_test_by_document, create_tensors, train

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

from sklearn.model_selection import train_test_split

In [14]:
dataset_path = '../preprosessing/all_attributes.json'
dataset = pd.read_json(dataset_path)
dataset.rename(index=str,columns={'Corpus':'texts'}, 
                inplace=True)
print(dataset.shape)
dataset.head()

(14042, 6)


,texts,Tag,I-ORG,I-MISC,I-PER,doc_ids
1,EU rejects German call to boycott British lamb .,I-ORG O I-MISC O O O I-MISC O O,"[[0, 2]]","[[11, 17], [34, 41]]",[],1
10,But Fischler agreed to review his proposal aft...,O I-PER O O O O O O O I-ORG O O O O O O O O O ...,"[[53, 55]]",[],"[[4, 12]]",1
100,Port conditions from Lloyds Shipping Intellige...,O O O I-ORG I-ORG I-ORG I-ORG O,"[[21, 27], [28, 36], [37, 49], [50, 57]]",[],[],9
1000,SOCCER - UEFA REWARDS THREE COUNTRIES FOR FAIR...,O O I-ORG O O O O O O O,"[[9, 13]]",[],[],60
10000,* Conglomerate Bollore lost 2.4 percent to 521...,O O I-ORG O O O O O O O O O O O O O O O O O O ...,"[[15, 22], [182, 186], [187, 193], [194, 203],...",[],[],595


In [15]:
attr_name = 'I-MISC'
#attr_name = 'CAD'
#attr_name = 'DIABETES'

selected_dataset = drop_noise_samples(dataset, attr_name)
selected_dataset.shape

(4489, 6)

In [16]:
#train_selected_dataset, test_selected_dataset = split_train_test_by_document(selected_dataset, test_ratio=0.2)

In [17]:
dataset_test_path = '../preprosessing/all_attributes_testb.json'
dataset_test = pd.read_json(dataset_test_path)
dataset_test.rename(index=str,columns={'Corpus':'texts'}, 
                inplace=True)
print(dataset_test.shape)
dataset_test.head()

(3455, 6)


,texts,Tag,I-ORG,I-MISC,I-PER,doc_ids
1,"SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRI...",O O I-LOC O O O O I-PER O O O O,[],[],"[[31, 36]]",1
10,Two goals from defensive errors in the last si...,O O O O O O O O O O O I-LOC O O O O O O O O O ...,[],[],[],1
100,Scoreboard in the second,O O O O,[],[],[],6
1001,INTERVIEW-ZYWIEC SEES NO BIG 97 NET RISE .,I-MISC O O O O O O O,[],"[[0, 16]]",[],55
1002,Steven Silber,I-PER I-PER,[],[],"[[0, 6], [7, 13]]",55


In [18]:
test_selected_dataset = drop_noise_samples(dataset_test, attr_name)
train_selected_dataset = selected_dataset
print('train', selected_dataset.shape)
print('test', test_selected_dataset.shape)

train (4489, 6)
test (1088, 6)


# Prepare model and preprocessing

In [19]:
from isanlp.en.processor_tokenizer_nltk_en import ProcessorTokenizerNltkEn

word_tokenizer = ProcessorTokenizerNltkEn()
bpe_tokenizer = BertTokenizer.from_pretrained('bert-base-cased', cache_dir=CACHE_DIR, do_lower_case=False)
train_sents, train_labels = tokenize_and_generate_labels(word_tokenizer, bpe_tokenizer, train_selected_dataset, attr_name, MAX_LEN)
test_sents, test_labels = tokenize_and_generate_labels(word_tokenizer, bpe_tokenizer, test_selected_dataset, attr_name, MAX_LEN)

In [20]:
lengths = np.array([len(sent) for sent in train_sents])
n_max_lengths = (lengths == MAX_LEN).sum()
print('N with max lengths:', n_max_lengths)
print('Ratio:', n_max_lengths / lengths.shape[0])

N with max lengths: 0
Ratio: 0.0


In [21]:
tags_vals = ['B', 'I', 'O', 'X', '[CLS]', '[SEP]']
tag2idx = {t : i for i, t in enumerate(tags_vals)}

In [22]:
train_input_ids, train_tags, train_attention_masks = create_tensors(bpe_tokenizer, tag2idx, 
                                                                    train_sents, train_labels, MAX_LEN)
test_input_ids, test_tags, test_attention_masks = create_tensors(bpe_tokenizer, tag2idx, 
                                                                 test_sents, test_labels, MAX_LEN)

In [23]:
def create_model_optimizer(tag2idx, full_finetuning=True):
    model = BertForTokenClassification.from_pretrained('bert-base-cased', 
                                                       cache_dir=CACHE_DIR, 
                                                       num_labels=len(tag2idx))
    model.cuda()
    
    #lr_head = 5e-4
    lr_head = 5e-2
    lr_body = 5e-5
    weight_decay = 0.01
    
    no_decay = ['bias', 'gamma', 'beta']

    logger.info(f'Full finetuning: {full_finetuning}')
    if full_finetuning:
        param_optimizer = list(model.bert.named_parameters())
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer 
                        if not any(nd in n for nd in no_decay)],
             'weight_decay': weight_decay},
            {'params': [p for n, p in param_optimizer 
                        if any(nd in n for nd in no_decay)],
             'weight_decay': 0.0},
            {'params' : [p for n, p in model.classifier.named_parameters()
                         if not any(nd in n for nd in no_decay)],
             'lr' : lr_head,
             'weight_decay': weight_decay},
            {'params' : [p for n, p in model.classifier.named_parameters()
                        if any(nd in n for nd in no_decay)],
             'lr' : lr_head,
             'weight_decay' : 0.0
            }
        ]
        
        n_params = sum(p.numel() for p in model.parameters())
    else:
        param_optimizer = list(model.classifier.named_parameters()) 
        optimizer_grouped_parameters = [
            {'params' : [p for n, p in param_optimizer
                         if not any(nd in n for nd in no_decay)],
             'lr' : lr_head,
             'weight_decay': weight_decay},
            {'params' : [p for n, p in param_optimizer
                         if any(nd in n for nd in no_decay)],
             'lr' : lr_head,
             'weight_decay' : 0.0
            }
        ]
    
        n_params = sum(p.numel() for p in model.classifier.parameters())
        
    logger.info(f'N parameters: {n_params}')

    optimizer = BertAdam(optimizer_grouped_parameters, lr=lr_body)
    #lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
    lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.7)
    
    return model, optimizer, lr_scheduler

# Training

In [24]:
from i2b2_utils import subsample_dataset
# Only for CAD and Diabetes
train_input_ids, train_tags, train_attention_masks = subsample_dataset(train_input_ids, train_tags, 
                                                                       train_attention_masks, 
                                                                       positive_tag=tag2idx['B'], 
                                                                       negative_ratio=0.2, positive_ratio=1.)

In [25]:
logger.info('Creating model...')
model, optimizer, lr_scheduler = create_model_optimizer(tag2idx, full_finetuning=True)
logger.info('Done.')

t_tr_inputs = torch.tensor(train_input_ids)
t_tr_tags = torch.tensor(train_tags)
t_tr_masks = torch.tensor(train_attention_masks)

t_val_inputs = torch.tensor(test_input_ids)
t_val_tags = torch.tensor(test_tags)
t_val_masks = torch.tensor(test_attention_masks)

train_data = TensorDataset(t_tr_inputs, t_tr_masks, t_tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, 
                              batch_size=BATCH_SIZE)

valid_data = TensorDataset(t_val_inputs, t_val_masks, t_val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, 
                              batch_size=PRED_BATCH_SIZE, shuffle=False)

train(model, optimizer, lr_scheduler, train_dataloader, valid_dataloader, 
      epochs=5, device=device, tags_vals=tags_vals)

2019-06-26 07:32:17,214 - active_learning - INFO - Creating model...
2019-06-26 07:32:25,645 - active_learning - INFO - Full finetuning: True
2019-06-26 07:32:25,649 - active_learning - INFO - N parameters: 108314886
2019-06-26 07:32:25,651 - active_learning - INFO - Done.


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

2019-06-26 07:32:59,140 - active_learning - INFO - Train loss: 0.17447211174294353
2019-06-26 07:33:03,340 - active_learning - INFO - Validation loss: 0.06728626787662506
2019-06-26 07:33:03,697 - active_learning - INFO - Validation F1-Score: 0.6149982059562255
2019-06-26 07:33:03,730 - active_learning - INFO - Validatin accuracy: 0.9896332050104547


Epoch:  20%|██        | 1/5 [00:38<02:32, 38.09s/it]

2019-06-26 07:33:37,241 - active_learning - INFO - Train loss: 0.011551058506630662
2019-06-26 07:33:41,245 - active_learning - INFO - Validation loss: 0.023718425072729588
2019-06-26 07:33:41,598 - active_learning - INFO - Validation F1-Score: 0.8019323671497585
2019-06-26 07:33:41,630 - active_learning - INFO - Validatin accuracy: 0.9963066460828953


...

Epoch:  60%|██████    | 3/5 [01:54<01:16, 38.03s/it]

2019-06-26 07:34:53,319 - active_learning - INFO - Train loss: 0.0011852914756731948
2019-06-26 07:34:57,337 - active_learning - INFO - Validation loss: 0.025049280375242233
2019-06-26 07:34:57,698 - active_learning - INFO - Validation F1-Score: 0.8109756097560976
2019-06-26 07:34:57,731 - active_learning - INFO - Validatin accuracy: 0.9962675629726614


Epoch:  80%|████████  | 4/5 [02:32<00:38, 38.04s/it]

2019-06-26 07:35:31,443 - active_learning - INFO - Train loss: 0.0008036139156942109
2019-06-26 07:35:35,465 - active_learning - INFO - Validation loss: 0.02563057281076908
2019-06-26 07:35:35,823 - active_learning - INFO - Validation F1-Score: 0.8183694530443757
2019-06-26 07:35:35,856 - active_learning - INFO - Validatin accuracy: 0.9964825200789479


Epoch: 100%|██████████| 5/5 [03:10<00:00, 38.07s/it]


# Evaluate

In [32]:
from i2b2_utils import evaluation_level_document, annotate_text 

pred_tags = annotate_text(model, valid_dataloader)
pos, pred_pos, tp = evaluation_level_document(pred_tags, test_selected_dataset, attr_name, tag2idx)

recall = tp / pos
precision = tp / pred_pos
f1 = 2. * recall * precision / (recall + precision)

print('Recall: ', recall)
print('Precision: ', precision)
print('F1:', f1)

Recall:  0.9835164835164835
Precision:  0.8647342995169082
F1: 0.9203084832904884


In [31]:
# Test A

In [ ]:
5 epochs
'I-ORG':
Recall:  0.968944099378882
Precision:  0.8342245989304813
F1: 0.896551724137931

In [ ]:
5 epochs
'I-PERS':
Recall:  1.0
Precision:  0.8743169398907104
F1: 0.9329446064139941

In [ ]:
5 epochs
'I_MISC'
Recall:  1.0
Precision:  0.946078431372549
F1: 0.9722921914357683

In [ ]:
# Test B

In [ ]:
5 epochs
'I_MISC' 
Recall:  0.9835164835164835
Precision:  0.8647342995169082
F1: 0.9203084832904884

In [3]:
torch.cuda.empty_cache()